In [1]:
!pip install -r requirements.txt

In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import pandas as pd
import openai
import os

In [4]:
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
df = pd.read_csv('fair-data-all.csv')
len(df)

556

In [6]:
df = df.drop_duplicates(subset=['Title'])
len(df)

555

In [7]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

@Language.factory("language_detector")
def get_lang_detector(nlp, name):
   return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('language_detector', last=True)

In [8]:
for index, row in df.iterrows():
    df.at[index, 'Lang'] = nlp(row['Title'])._.language['language']

In [9]:
df = df[df.Lang == 'en']
len(df)

549

In [123]:
df.iloc[[62,355,406,464]].DOI

62            10.1162/dint_x_00187
358    10.1016/j.jphys.2017.02.002
410           10.2147/clep.s105933
469     10.1186/s12939-023-01855-0
Name: DOI, dtype: object

In [10]:
df2 = df[df['Abstract'].str.contains('FAIR')].drop(62).drop(355).drop(406).drop(464)
len(df2)

31

In [95]:
intro = "You need to decide in which category the following text falls into \
according to its content. Possible choices are: \
'Overview and Adoption of FAIR Principles' (that could include general reviews and surveys, case studies, guidelines, challenges and barriers to adoption),\
'FAIRification of data' (that could include methodologies for data storage, access, curation, preservation, archiving, security, privacy),\
'Infrastructure and Tools' (that could include data repositories and platforms, metadata standards and ontologies).\n\nText: "

In [96]:
for index, row in df2.iterrows():
    text = intro + row['Abstract'] + "\nCategory:"
    response = openai.Completion.create(model="text-davinci-003", 
                                    prompt=text, 
                                    temperature=0, 
                                    max_tokens=20)

    df2.at[index, 'Category'] = response["choices"][0]["text"].strip()

In [97]:
df2['Category'].value_counts()

Overview and Adoption of FAIR Principles    18
Infrastructure and Tools                     8
FAIRification of data                        5
Name: Category, dtype: int64

In [99]:
df2[['Category', 'DOI', 'Title']].sort_values(['Category','Title'])

,Category,DOI,Title
2,FAIRification of data,10.1016/j.dib.2020.106424,An Exploratory Assessment of a Multidimensiona...
326,FAIRification of data,10.1186/s13326-022-00263-7,Applying the FAIR principles to data in a hosp...
515,FAIRification of data,10.23889/ijpds.v7i3.1887,Augmenting laboratory COVID serology data gran...
187,FAIRification of data,10.1162/dint_a_00182,FAIR Machine Learning Model Pipeline Implement...
124,FAIRification of data,10.1371/journal.pone.0212291,Term sets: A transparent and reproducible repr...
87,Infrastructure and Tools,10.2196/42822,A Data Transformation Methodology to Create Fi...
524,Infrastructure and Tools,10.1093/eurpub/ckac129.468,An enhanced version of the PHIRI infrastructur...
456,Infrastructure and Tools,10.1038/s41431-020-0637-5,"Connecting data, tools and people across Europ..."
166,Infrastructure and Tools,10.1016/j.radonc.2019.11.019,Distributed learning on 20 000+ lung cancer pa...
505,Infrastructure and Tools,10.1038/s41597-022-01265-x,FAIR Genomes metadata schema promoting Next Ge...


In [124]:
df2.to_csv('fair-data-only.csv')

In [101]:
df2['Abstract'].str.contains('privacy|security', regex=True).sum()

5

In [102]:
df3 = df2[df2['Abstract'].str.contains('privacy|security', regex=True)]

In [103]:
df3.DOI

36              10.3390/app12094336
87                    10.2196/42822
147       10.1016/j.jbi.2020.103421
166    10.1016/j.radonc.2019.11.019
237            10.15265/iy-2017-024
Name: DOI, dtype: object

In [6]:
df2 = pd.read_csv('fair-data-only.csv')
len(df2)

31

In [7]:
df4 = df2.loc[df2['Category'] == 'Overview and Adoption of FAIR Principles', ['DOI', 'Title', 'Abstract']]
len(df4)

18

In [8]:
pd.set_option('display.max_colwidth', None)

In [9]:
intro = "Explain the role of FAIR principles in the following text. Use only three sentences. Do not tell what the FAIR principles are.\n\nText: "

In [10]:
for index, row in df4.iterrows():
    text = intro + row['Abstract'] + "\nSummary:"
    response = openai.Completion.create(model="text-davinci-003", 
                                    prompt=text, 
                                    temperature=0, 
                                    max_tokens=300)

    df4.at[index, 'Summary'] = response["choices"][0]["text"].strip()

In [13]:
df4.to_csv('fair-data-summary.csv')

In [14]:
df4[['DOI', 'Summary']]

,DOI,Summary
1,10.1016/j.ejmp.2021.01.083,"The FAIR principles are discussed in this article in relation to the use of artificial intelligence in radiotherapy. The article outlines the need for and usage of the FAIR principles, and provides guidance and potential applications of FAIR to various radiotherapy stakeholders. It emphasizes the importance of medical physicists in this process."
4,10.1016/j.clon.2021.11.040,"The FAIR principles are used as a framework to discuss the structural reasons why artificial intelligence has not yet been able to make full use of existing datasets. The principles are used to identify barriers preventing artificial intelligence from making use of existing datasets, and to suggest ways to enable progress. The Brain Tumour Data Accelerator is an example of how the FAIR principles can be used to integrate fragmented patient data into an enriched dataset."
7,10.1016/j.drudis.2022.01.006,"The FAIR principles are being used to enable a shift from an application-centric to a data-centric perspective in the life sciences. This shift is necessary to maximize the value of data generated from research, clinical trials, and real-world healthcare data. Use cases for FAIR implementation are being considered alongside assessment of data quality to achieve this goal."
8,10.1136/bmj.i4340,"The FAIR Health non-profit corporation conducted an analysis of privately billed healthcare claims to show the rise of opioid dependence in the United States. The FAIR principles ensure that the data collected is accurate, transparent, and accessible to all stakeholders. This data is then used to inform government agencies, insurers, providers, and consumers of the current opioid epidemic in the US."
9,10.1016/j.jbi.2020.103421,"The FAIR principles are a set of guidelines that aim to ensure that data is Findable, Accessible, Interoperable, and Reusable. The text discusses the challenges of data sharing and interoperability, and how the lack of standards and traceable semantics can make it difficult to adhere to the FAIR principles. The text also suggests that common data elements (CDEs) could be used to standardize data collection and enable responsible data sharing that would help to adhere to the FAIR principles."
10,10.3233/shti210801,"The FAIR principles are used in this paper to ensure data reuse and improve image classification performance in imbalanced datasets. The proposed framework is tested on an acne disease dataset and is shown to outperform the baselines, with up to 5% improvement in image classification. The FAIR principles are essential for ensuring data reuse and accuracy in medical image classification and diagnosis."
12,10.1136/bmjos-2021-100268,The FAIR principles are used as the basis for this study to bridge gaps in the management and sharing of NGS data. The analysis is part of a programme to develop an instruction manual for FAIR genomic data in clinical care and research. The gap analysis is a possible contribution from the Netherlands to the European 1+ Million Genomes Initiative.
14,10.15265/iy-2017-024,The FAIR Guiding Principles are used to evaluate the risk of privacy breaches in releasing genomics datasets. They are also used to create phenotype models via a machine learning process and to securely connect ResearchKit compatible applications to the widely-used clinical research infrastructure Informatics for Integrating Biology and the Bedside (i2b2). The FAIR principles are essential for ensuring the safe and secure use of data in the field of Clinical Research Informatics.
15,10.3390/ijerph15081644,"The FAIR principles are used as a framework to improve the quality of RD registries. This includes aspects of governance, data and information, infrastructure, documentation, training, and quality audit. The list of recommendations is intended to be used by established and new RD registries, and a toolkit is being developed to enable continuous assessment and improvement."
17,10.1515/cclm-2022-1096